In [1]:

!git clone https://github.com/neurodata/SPORF.git

%cd /content/SPORF/Python

!apt-get update
!apt-get install build-essential cmake python3-dev libomp-dev libeigen3-dev   # Ubuntu/Debian


!python setup.py clean --all

!pip install -e .

fatal: destination path 'SPORF' already exists and is not an empty directory.
/content/SPORF/Python
Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.e

In [2]:
'''
REGRESSION:
    Goal: Predict Productivity Score based on features

'''

'\nREGRESSION:\n    Goal: Predict Productivity Score based on features\n\n'

In [5]:
import pandas as pd

df = pd.read_csv('/content/social_media_vs_productivity_cleaned.csv')

In [6]:
df.columns

Index(['Unnamed: 0', 'age', 'gender', 'job_type', 'daily_social_media_time',
       'social_platform_preference', 'number_of_notifications',
       'work_hours_per_day', 'perceived_productivity_score',
       'actual_productivity_score', 'stress_level', 'sleep_hours',
       'screen_time_before_sleep', 'breaks_during_work', 'uses_focus_apps',
       'has_digital_wellbeing_enabled', 'coffee_consumption_per_day',
       'days_feeling_burnout_per_month', 'weekly_offline_hours',
       'job_satisfaction_score'],
      dtype='object')

In [7]:
# features that don't have anything to do with productivity, as well as the output.

input_invalid_columns =  ['social_platform_preference', 'perceived_productivity_score', 'actual_productivity_score','Unnamed: 0']

In [8]:
features = df.drop(columns=input_invalid_columns)

features.head()

,age,gender,job_type,daily_social_media_time,number_of_notifications,work_hours_per_day,stress_level,sleep_hours,screen_time_before_sleep,breaks_during_work,uses_focus_apps,has_digital_wellbeing_enabled,coffee_consumption_per_day,days_feeling_burnout_per_month,weekly_offline_hours,job_satisfaction_score
0,56,0,0,4.180940,61,6.753558,4.0,5.116546,0.419102,8,0,0,4,11,21.927072,6.336688
1,46,0,1,3.249603,59,9.169296,7.0,5.103897,0.671519,7,1,1,2,25,0.000000,3.412427
2,32,0,2,0.000000,57,7.910952,4.0,8.583222,0.624378,0,1,0,3,17,10.322044,2.474944
3,60,1,0,0.000000,59,6.355027,6.0,6.052984,1.204540,1,0,0,0,4,23.876616,1.733670
4,25,0,3,0.000000,66,6.214096,7.0,5.405706,1.876254,1,0,1,1,30,10.653519,9.693060


In [9]:
output = df['actual_productivity_score']

In [10]:
import numpy as np

input_features = features.to_numpy()
outputs = output.to_numpy()

In [11]:
from sklearn.model_selection import train_test_split

# First split: train vs test
X_train_full, X_test, y_train_full, y_test = train_test_split(
    input_features, outputs, test_size=0.2, random_state=42
)

# Second split: train vs validation
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.25, random_state=42
)


In [12]:
# shapes

print(f"X_train shape {X_train.shape}")
print(f"X_test shape {X_test.shape}")
print(f"y_train shape {y_train.shape}")
print(f"y_test shape {y_test.shape}")

print("validation:")
print(f"X_val shape {X_val.shape}")
print(f"y_val shape {y_val.shape}")

X_train shape (18000, 16)
X_test shape (6000, 16)
y_train shape (18000,)
y_test shape (6000,)
validation:
X_val shape (6000, 16)
y_val shape (6000,)


In [13]:
from rerf.RerF import fastPredict,fastPredictPost, fastRerF
from multiprocessing import cpu_count

In [14]:

forest = fastRerF(
    X=X_train,
    Y=y_train,
    forestType="rfBase",   # regression forest
    trees=500,
    maxDepth=20,
    minParent=5,
    numCores=cpu_count()
)

forest.printParameters()

numTreesInForest -> 500
maxDepth -> 10
minParent -> 5
numClasses -> 10
numObservations -> 18000
numFeatures -> 16
mtry -> 4
mtryMult -> 1.5
fractionOfFeaturesToTest -> -1
CSV file name -> 
columnWithY -> -1
Type of Forest -> rfBase
binSize -> 0
binMin -> 0
numCores -> 2
seed -> 105150
numTreeBins -> -1


In [15]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Predict using your custom function
train_pred = fastPredict(X_train, forest)
test_pred = fastPredict(X_test, forest)

# Compute final errors
train_rmse = mean_squared_error(y_train, train_pred)  # RMSE
test_rmse = mean_squared_error(y_test, test_pred)

train_mae = mean_absolute_error(y_train, train_pred)  # MAE
test_mae = mean_absolute_error(y_test, test_pred)

print("Train RMSE:", train_rmse)
print("Test RMSE:", test_rmse)
print("Train MAE:", train_mae)
print("Test MAE:", test_mae)


Train RMSE: 3.6679694292175222
Test RMSE: 3.793358405539604
Train MAE: 1.2870239391790497
Test MAE: 1.3305211967776167


In [16]:
import pandas as pd

# Basic stats
print("Min:", outputs.min())
print("Max:", outputs.max())
print("Range:", outputs.max() - outputs.min())
print("Mean:", outputs.mean())
print("Std:", outputs.std())



Min: 0.0
Max: 9.84625801321986
Range: 9.84625801321986
Mean: 4.561437594985104
Std: 2.2467754361556245
